In [4]:
import numpy as np
import pandas as pd
from collections import OrderedDict

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

In [5]:
data = pd.read_csv("./data/processed_data.csv")

In [6]:
data.head()

,time,latitude,longitude,surface,u10,v10,d2m,t2m,sp,tcc,...,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_6,hour_12,hour_18
0,2010-01-01 00:00:00,53.5,3.50,0.0,-8.545227,-7.018692,270.34955,277.60290,100182.81,0.651825,...,False,False,False,False,False,False,True,False,False,False
1,2010-01-01 00:00:00,53.5,3.75,0.0,-8.562805,-6.929825,270.52728,277.48962,100171.81,0.710113,...,False,False,False,False,False,False,True,False,False,False
2,2010-01-01 00:00:00,53.5,4.00,0.0,-8.588196,-6.831192,270.71674,277.33142,100167.81,0.787964,...,False,False,False,False,False,False,True,False,False,False
3,2010-01-01 00:00:00,53.5,4.25,0.0,-8.690735,-6.696426,270.91595,277.14783,100167.81,0.839111,...,False,False,False,False,False,False,True,False,False,False
4,2010-01-01 00:00:00,53.5,4.50,0.0,-8.834290,-6.563614,271.01752,276.96228,100161.81,0.888733,...,False,False,False,False,False,False,True,False,False,False


In [122]:
data_ams = data[(data['latitude'] == 52.25) & (data['longitude'] == 5.0)]
data_ams.shape

(13148, 29)

In [123]:
data_ams['time'] = pd.to_datetime(data_ams['time'], format='%Y-%m-%d %H:%M:%S')

data_ams['days'] = (pd.Timestamp('20220101') - data_ams['time']).dt.days
data_ams['month'] = data_ams['time'].dt.month
data_ams['hour'] = data_ams['time'].dt.hour

data_ams = data_ams.drop(['time', 'latitude', 'longitude', 'surface', 'step', 'number', 'valid_time', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'hour_0', 'hour_6', 'hour_12', 'hour_18'], axis=1)

/var/folders/zm/_f1r7jt5567946w_k48x461h0000gn/T/ipykernel_11127/2575447760.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ams['time'] = pd.to_datetime(data_ams['time'], format='%Y-%m-%d %H:%M:%S')
/var/folders/zm/_f1r7jt5567946w_k48x461h0000gn/T/ipykernel_11127/2575447760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ams['days'] = (pd.Timestamp('20220101') - data_ams['time']).dt.days
/var/folders/zm/_f1r7jt5567946w_k48x461h0000gn/T/ipykernel_11127/2575447760.py:4: SettingWithCopyWarnin

In [148]:
data_ams.tail(10)

,u10,v10,d2m,t2m,sp,tcc,days,month,hour
2561991,5.761032,4.689850,280.46973,281.60498,102726.125,1.000000,1098,12,12
2562186,7.464996,-3.053085,279.80566,282.40870,102639.810,0.879791,1098,12,18
2562381,4.699738,-5.373718,278.53485,281.30896,103059.190,0.977051,1098,12,0
2562576,2.219101,-1.308884,278.38495,279.75793,103276.875,0.994232,1097,12,6
2562771,3.400070,1.338898,280.44153,282.42432,103271.125,1.000000,1097,12,12
2562966,2.763321,-0.800598,282.38745,282.43860,103198.500,1.000000,1097,12,18
2563161,3.318680,-1.437378,282.03888,282.47020,103237.750,0.989960,1097,12,0
2563356,3.041580,-1.872452,281.70374,282.27515,103336.000,0.992493,1096,12,6
2563551,3.735977,-1.097519,281.94330,282.89575,103397.875,1.000000,1096,12,12
2563746,4.349091,1.064712,280.63640,281.78625,103343.375,0.982666,1096,12,18


In [154]:
# Prepare dataset

def prepare_data(raw_data, interval):
    X, y = [], []
    for i in range(interval, len(raw_data)):
        X.append(raw_data.iloc[i - interval:i, :])
        y.append(raw_data.iloc[i, 3])
    return np.array(X), np.array(y)

In [155]:
X, y = prepare_data(data_ams, 30)

X.shape, y.shape

((13118, 30, 9), (13118,))

In [151]:
y[-10:]

array([281.60498, 282.4087 , 281.30896, 279.75793, 282.42432, 282.4386 ,
       282.4702 , 282.27515, 282.89575, 281.78625])

In [20]:
# X = data_ams.drop(['t2m'], axis=1)
# y = data_ams['t2m']

# X.shape, y.shape

((13148, 6), (13148,))

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X.reshape(X.shape[0], -1), y, test_size=0.2, random_state=0)

In [157]:
X_train.shape, y_train.shape

((10494, 270), (10494,))

In [158]:
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "warm_start":True,
    "oob_score":True,
    "random_state": 42,
}

In [159]:
reg = RandomForestRegressor(**params)

In [160]:
reg.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, min_samples_split=5, n_estimators=500,
                      oob_score=True, random_state=42, warm_start=True)

In [161]:
y_pred = reg.predict(X_test)

In [162]:
# Evaluation
mse = mean_squared_error(y_test, y_pred)
train_score = round(reg.score(X_train, y_train) * 100, 2)
valid_score = round(reg.score(X_test, y_test) * 100, 2)

print('Mean Squared Error (MSE):', mse)

Mean Squared Error (MSE): 4.874189107778368
